### GAN
**- 실제 이미지를 주고, 구분자에게 이 이미지 진짜임을 판단하게 함 <br/> - 생성자를 통해 노이즈로부터 임의의 이미지 생성<br/> - 그 이미지를 다시 같은 구분자를 통해 진짜인지 판단하게 함<br/>**
→ 생성자와 구분자의 경쟁을 통해 생성자는 실제 이미지와 상당히 비슷한 이미지 생성

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
# 생성자의 입력값으로 사용할 노이즈 크기
n_noise = 128

In [3]:
X = tf.placeholder(tf.float32, [None, n_input])
# 노이즈를 입력할 플레이스홀더 Z (가짜 이미지는 노이즈에서 생성) 
Z = tf.placeholder(tf.float32, [None, n_noise])

In [5]:
# 생성자 신경망에 사용할 변수
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [6]:
# 구분자 신경망에 사용할 변수
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [7]:
# 생성자
# 실제 이미지와 같은 크기의 결과값 출력
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    
    return output

In [8]:
# 구분자
# 0~1 사이의 스칼라값 하나 출력
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    
    return output

In [17]:
# 무작위한 노이즈
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [10]:
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

In [11]:
# 손실값 - 경찰 학습용
# D_real 은 1에 가까워야함 (진짜라고 판별)
# D_gene 은 0에 가까워야함 (가짜라고 판별)
# loss_D 값을 최대화
loss_D = tf.reduce_mean(tf.log(D_real)+tf.log(1-D_gene))

In [12]:
# 손실값 - 위조지폐범 학습용
# D_gene 은 1에 가까워야함 (가짜이미지를 넣어도 진짜같다고 판별)
# loss_G 값을 최대화
# loss_D와 loss_G는 서로 증가하면 하락하는 경쟁관계
loss_G = tf.reduce_mean(tf.log(D_gene))

In [13]:
# loss_D를 구할때는 구분자 신경망에 사용되는 변수들만 사용
# loss_G를 구할때는 생성자 신경망에 사용되는 변수들만 사용
# 그래야 loss_D 학습할땐 생성자 변하지 않고, loss_G 학습할땐 구분자 변하지 않음
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [14]:
# loss를 최대화 해야하는데 최적화에 쓸 수 있는 함수는 minimize뿐임
# 따라서 loss_D와 loss_G에 음수 부호 붙이기
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

In [18]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')

Epoch: 0000 D loss: -0.5022 G loss: -2.03
Epoch: 0001 D loss: -0.3032 G loss: -2.573
Epoch: 0002 D loss: -0.199 G loss: -2.715
Epoch: 0003 D loss: -0.4047 G loss: -1.729
Epoch: 0004 D loss: -0.2652 G loss: -2.386
Epoch: 0005 D loss: -0.3727 G loss: -2.302
Epoch: 0006 D loss: -0.5898 G loss: -1.817
Epoch: 0007 D loss: -0.4333 G loss: -2.221
Epoch: 0008 D loss: -0.3463 G loss: -2.11
Epoch: 0009 D loss: -0.2463 G loss: -2.692
Epoch: 0010 D loss: -0.241 G loss: -2.612
Epoch: 0011 D loss: -0.3913 G loss: -2.667
Epoch: 0012 D loss: -0.3514 G loss: -2.381
Epoch: 0013 D loss: -0.3553 G loss: -2.527
Epoch: 0014 D loss: -0.2548 G loss: -2.53
Epoch: 0015 D loss: -0.2696 G loss: -2.644
Epoch: 0016 D loss: -0.377 G loss: -2.424
Epoch: 0017 D loss: -0.5054 G loss: -2.178
Epoch: 0018 D loss: -0.4251 G loss: -2.407
Epoch: 0019 D loss: -0.4149 G loss: -2.44
Epoch: 0020 D loss: -0.3796 G loss: -2.622
Epoch: 0021 D loss: -0.5134 G loss: -2.337
Epoch: 0022 D loss: -0.5686 G loss: -2.183
Epoch: 0023 D loss

In [2]:
#####
# 원하는 숫자 지정해 생성하는 모델
#####

total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

In [3]:
X = tf.placeholder(tf.float32, [None, n_input])
# 노이즈와 실제 이미지에 각각에 해당하는 숫자를 힌트로 넣어주는 용도 Y
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [4]:
# 생성자 신경망
def generator(noise, labels):
    with tf.variable_scope('generator'):
        inputs = tf.concat([noise, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input, activation=tf.nn.sigmoid)
        
    return output

In [5]:
# 구분자 신경망
def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        # 이전에 사용한 변수 재사용
        # 진짜 이미지를 판별할때와 가짜 이미지를 판별할 때 똑같은 변수 사용해야함
        if reuse:
            scope.reuse_variables()
        
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, 1, activation=None)

    return output        

In [6]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

In [7]:
G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

In [8]:
# D_real 값을 1에 가깝도록 하는 손실함수
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real, labels=tf.ones_like(D_real)))
# D_gene 값을 0에 가깝도록 하는 손실함수
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.zeros_like(D_gene)))
# loss_D 값을 최소화하여 구분자 학습
loss_D = loss_D_real + loss_D_gene

In [9]:
# D_gene 값을 1에 가깝도록 하는 손실함수
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [10]:
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Y: batch_ys, Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G,feed_dict={Y: mnist.test.labels[:sample_size], Z: noise})

        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))

        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()

            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')


Epoch: 0000 D loss: 0.0123 G loss: 7.933
Epoch: 0001 D loss: 0.02002 G loss: 7.485
Epoch: 0002 D loss: 0.01276 G loss: 7.077
Epoch: 0003 D loss: 0.02375 G loss: 6.731
Epoch: 0004 D loss: 0.02979 G loss: 7.262
Epoch: 0005 D loss: 0.0331 G loss: 8.376
Epoch: 0006 D loss: 0.04067 G loss: 7.367
Epoch: 0007 D loss: 0.03982 G loss: 6.857
Epoch: 0008 D loss: 0.103 G loss: 7.613
Epoch: 0009 D loss: 0.07835 G loss: 6.981
Epoch: 0010 D loss: 0.1329 G loss: 5.463
Epoch: 0011 D loss: 0.2091 G loss: 4.485
Epoch: 0012 D loss: 0.2884 G loss: 4.908
Epoch: 0013 D loss: 0.3033 G loss: 4.911
Epoch: 0014 D loss: 0.5238 G loss: 4.477
Epoch: 0015 D loss: 0.5375 G loss: 4.274
Epoch: 0016 D loss: 0.4465 G loss: 3.665
Epoch: 0017 D loss: 0.5388 G loss: 3.543
Epoch: 0018 D loss: 0.952 G loss: 3.388
Epoch: 0019 D loss: 0.6796 G loss: 3.44
Epoch: 0020 D loss: 0.5874 G loss: 2.998
Epoch: 0021 D loss: 0.5679 G loss: 2.797
Epoch: 0022 D loss: 0.8099 G loss: 3.026
Epoch: 0023 D loss: 0.4972 G loss: 3.088
Epoch: 0024 